# Imports

In [39]:
import numpy as numpy
import pandas as pd

from darts import TimeSeries
from darts.models import TransformerModel
from darts.metrics import mae, mape
from powderalert.ml_logic.data import fetch_weather_data

import optuna

# Improving Model Performance | Target = Snowfall

## Getting Started | Import basic dataset & preprocess

In [ ]:
# for future reference - get data directly from api?
# fetch_weather_data()

In [26]:
# in the meantime use preprocessed csv (refer to notebook_Anita-Gei_get_preprocessed_data.ipynb for more info)

df = pd.read_csv('/home/anita/code/MadMax1995bb/powder_alert2.0/raw_data/historical_data_preprocessed.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.drop(columns='Unnamed: 0')
df = df.set_index('date')
df.tail(2)

,snowfall,weather_code_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,sunshine_duration,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,0.0,3.0,-1.911309,0.072451,-1.882762,-0.389763,-0.272938,1.403548,-0.129623,-1.404447,0.905132,-1.084686,-0.925955,1.644208,-0.619531,-0.580998,0.825689,0.846788,-0.413650,-0.063053,-0.097270,-0.565147,-0.500000,0.866025,0.0,1.0,0.0,1.0
2024-01-01 23:00:00,0.0,1.0,-1.875245,-0.040464,-1.882762,-0.389763,-0.272938,1.403548,-0.280686,-1.495166,-0.719864,-1.084686,-0.978679,0.135490,-0.619531,-0.561763,0.696605,0.882111,-0.427446,-0.142219,-0.044957,-0.565147,-0.258819,0.965926,0.0,1.0,0.0,1.0


In [27]:
print(df.shape)

(131496, 28)


## Feature Engineering

In [28]:
# Create lag features for snowfall and precipitation
lags = [1, 3, 6, 12]
for lag in lags:
    df[f'snowfall_lag_{lag}'] = df['snowfall'].shift(lag)
    df[f'precipitation_lag_{lag}'] = df['precipitation'].shift(lag)

df.tail(2)

,snowfall,weather_code_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,sunshine_duration,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,0.0,3.0,-1.911309,0.072451,-1.882762,-0.389763,-0.272938,1.403548,-0.129623,-1.404447,0.905132,-1.084686,-0.925955,1.644208,-0.619531,-0.580998,0.825689,0.846788,-0.413650,-0.063053,-0.097270,-0.565147,-0.500000,0.866025,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763
2024-01-01 23:00:00,0.0,1.0,-1.875245,-0.040464,-1.882762,-0.389763,-0.272938,1.403548,-0.280686,-1.495166,-0.719864,-1.084686,-0.978679,0.135490,-0.619531,-0.561763,0.696605,0.882111,-0.427446,-0.142219,-0.044957,-0.565147,-0.258819,0.965926,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763


In [29]:
# Add rolling statistics
df['cloud_cover_rolling_mean_3h'] = df['cloud_cover'].rolling(window=3).mean()
df['precipitation_rolling_std_6h'] = df['precipitation'].rolling(window=6).std()
df.tail(2)

,snowfall,weather_code_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,sunshine_duration,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,0.0,3.0,-1.911309,0.072451,-1.882762,-0.389763,-0.272938,1.403548,-0.129623,-1.404447,0.905132,-1.084686,-0.925955,1.644208,-0.619531,-0.580998,0.825689,0.846788,-0.413650,-0.063053,-0.097270,-0.565147,-0.500000,0.866025,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.182911,0.0
2024-01-01 23:00:00,0.0,1.0,-1.875245,-0.040464,-1.882762,-0.389763,-0.272938,1.403548,-0.280686,-1.495166,-0.719864,-1.084686,-0.978679,0.135490,-0.619531,-0.561763,0.696605,0.882111,-0.427446,-0.142219,-0.044957,-0.565147,-0.258819,0.965926,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.309300,0.0


In [30]:
# Drop rows with NaN values (introduced by lags and rolling)
df = df.dropna()
print(df.shape)

(131484, 38)


## Use Subset of Data while finding the "best" Model!

In [36]:
train_df = df[-50000:]
print(train_df.shape)
train_df.tail(2)

(50000, 38)


,snowfall,weather_code_encoded,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snow_depth,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,sunshine_duration,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,month_sin,month_cos,snowfall_lag_1,precipitation_lag_1,snowfall_lag_3,precipitation_lag_3,snowfall_lag_6,precipitation_lag_6,snowfall_lag_12,precipitation_lag_12,cloud_cover_rolling_mean_3h,precipitation_rolling_std_6h
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-01 22:00:00,0.0,3.0,-1.911309,0.072451,-1.882762,-0.389763,-0.272938,1.403548,-0.129623,-1.404447,0.905132,-1.084686,-0.925955,1.644208,-0.619531,-0.580998,0.825689,0.846788,-0.413650,-0.063053,-0.097270,-0.565147,-0.500000,0.866025,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.182911,0.0
2024-01-01 23:00:00,0.0,1.0,-1.875245,-0.040464,-1.882762,-0.389763,-0.272938,1.403548,-0.280686,-1.495166,-0.719864,-1.084686,-0.978679,0.135490,-0.619531,-0.561763,0.696605,0.882111,-0.427446,-0.142219,-0.044957,-0.565147,-0.258819,0.965926,0.0,1.0,0.0,1.0,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.0,-0.389763,0.309300,0.0


## Create Time Series

In [37]:
feature_columns = train_df.drop(columns=['snowfall']).columns.tolist()

snowfall_series = TimeSeries.from_dataframe(train_df, value_cols='snowfall')
feature_series = TimeSeries.from_dataframe(train_df, value_cols=feature_columns)

## Split into Train, Validation and Test Sets

In [38]:
y_train_val, y_test = snowfall_series.split_before(0.8)
y_train, y_val = y_train_val.split_before(0.8)

X_train_val, X_test = feature_series.split_before(0.8)
X_train, X_val = X_train_val.split_before(0.8)

## Optimize Hyperparamters with Optuna

In [ ]:
def objective(trial):
    d_model = trial.suggest_int("d_model", 32, 128)
    nhead = trial.suggest_int("nhead", 2, 8)
    dropout = trial.suggest_uniform("dropout", 0.01, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

    model = TransformerModel(
        input_chunk_length=48,
        output_chunk_length=48,
        batch_size=batch_size,
        n_epochs=20,  # Reduced for faster tuning
        model_name="transformer_snowfall_optuna",
        nr_epochs_val_period=1,
        d_model=d_model,
        nhead=nhead,
        num_encoder_layers=2,
        num_decoder_layers=2,
        dropout=dropout,
        random_state=42
    )

    model.fit(
        series=y_train,
        past_covariates=X_train,
        val_series=y_val,
        val_past_covariates=X_val,
        verbose=False
    )

    predictions = model.predict(n=48, past_covariates=X_val)
    return mae(y_val, predictions)

In [ ]:
# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

In [ ]:
# Print best parameters
print(study.best_value)
print(f"Best trial: {study.best_trial.params}")

## Initialize Model with optimized Parameters

In [ ]:
#fill after completing optuna study!
model = TransformerModel(
    input_chunk_length=48,
    output_chunk_length=48,
    batch_size=32,
    n_epochs=20,
    model_name="snowfall_model",
    nr_epochs_val_period=1,
    d_model=64,
    nhead=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    dropout=0.1,
    random_state=42,
    likelihood=None  # Regression task
)


## Fit Model

In [ ]:
model.fit(
    series=y_train,
    past_covariates=X_train,
    val_series=y_val,
    val_past_covariates=X_val,
    verbose=True
)

## Make Predictions

In [ ]:
forecast = model.predict(n=48, past_covariates=X_test)